In [14]:
import datetime
import ifcopenshell.api.sequence
from ifcopenshell.util.element import get_decomposition
from ifcopenshell.util.placement import get_storey_elevation

# Define a convenience function to add a task chained to a predecessor
def add_task(model, name, predecessor, work_schedule):
    # Add a construction task
    task = ifcopenshell.api.sequence.add_task(model,
        work_schedule=work_schedule, name=name, predefined_type="CONSTRUCTION")

    # Give it a time
    task_time = ifcopenshell.api.sequence.add_task_time(model, task=task)

    # Arbitrarily set the task's scheduled time duration to be 1 week
    ifcopenshell.api.sequence.edit_task_time(model, task_time=task_time,
        attributes={"ScheduleStart": datetime.date(2000, 1, 1), "ScheduleDuration": "P1W"})

    # If a predecessor exists, create a finish to start relationship
    if predecessor:
        ifcopenshell.api.sequence.assign_sequence(model, relating_process=predecessor, related_process=task)

    return task

# Open an existing IFC4 model you have of a building
model = ifcopenshell.open("sample_models/CASA TIPO 11111.ifc")

# Create a new construction schedule
schedule = ifcopenshell.api.sequence.add_work_schedule(model, name="Construction")

# Let's imagine a starting task for site establishment.
task = add_task(model, "Site establishment", None, schedule)
start_task = task

# Get all our storeys sorted by elevation ascending.
storeys = sorted(model.by_type("IfcBuildingStorey"), key=lambda s: get_storey_elevation(s))

# For each storey ...
for storey in storeys:

    # Add a construction task to construct that storey, using our convenience function
    task = add_task(model, f"Construct {storey.Name}", task, schedule)

    # Assign all the products in that storey to the task as construction outputs.
    for product in get_decomposition(storey):
        ifcopenshell.api.sequence.assign_product(model, relating_product=product, related_object=task)

# Ask the computer to calculate all the dates for us from the start task.
# For example, if the first task started on the 1st of January and took a
# week, the next task will start on the 8th of January. This saves us
# manually doing date calculations.
ifcopenshell.api.sequence.cascade_schedule(model, task=start_task)

# Calculate the critical path and floats.
ifcopenshell.api.sequence.recalculate_schedule(model, work_schedule=schedule)

# Write out to a file
model.write("model.ifc")

In [15]:
import ifcopenshell
import ifcopenshell.api.root

def add_structural_analysis_model(file: ifcopenshell.file) -> ifcopenshell.entity_instance:
    """Add a new structural analysis model

    A structural analysis model is a group of all the loads, reactions,
    structural members, and structural connections required to describe a
    structural analysis model.

    A 3D analytical model is assumed.

    :return: The newly created IfcStructuralAnalysisModel
    :rtype: ifcopenshell.entity_instance
    """
    return ifcopenshell.api.root.create_entity(
        file, ifc_class="IfcStructuralAnalysisModel", predefined_type="LOADING_3D"
    )

# Path to your existing IFC file
existing_ifc_file_path = "sample_models/CASA TIPO 11111.ifc"

# Open the existing IFC file
file = ifcopenshell.open(existing_ifc_file_path)

# Add a structural analysis model to the file
structural_analysis_model = add_structural_analysis_model(file)

# Print the created entity to verify
print("Created Structural Analysis Model:", structural_analysis_model)

# Save the modified file to disk
file.write("modified_structural_analysis_model.ifc")

print("Structural analysis model added and file saved as 'modified_structural_analysis_model.ifc'.")


Created Structural Analysis Model: #4085=IfcStructuralAnalysisModel('3_7npPbtDE$xSVZERDEiEE',#4084,$,$,$,.LOADING_3D.,$,$,$,$)
Structural analysis model added and file saved as 'modified_structural_analysis_model.ifc'.
